In [ ]:
%%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
from sklearn.metrics import accuracy_score
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer.padding_side = "left"
model = model.to("cuda")

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


In [ ]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# print and see dataset
dataset

In [ ]:
prompt = """### Example:

Problem:
Find the value of y if 4y - 7 = 9.

Solution:
Adding 7 to both sides, we get 4y = 16. Then, dividing by 4, we find y = 4.

Answer:
4

Output:
True

Now, evaluate the following problem:

### Problem:
{}

### Solution:
{}

### Answer:
{}

### Output:
{}"""



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    output      = examples["is_correct"]
    explaination = examples["solution"] #my edit
    texts = []
    for instruction, input_explaination, input, output in zip(question, explaination, ans, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input_explaination, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }




In [ ]:
from sklearn.metrics import accuracy_score

# Sample 110,000 random rows from the original dataset
sampled_data = dataset['train'].shuffle(seed=3407).select(range(110000))

# Split the sampled data into 100,000 for training and 10000 for testing
split_data = sampled_data.train_test_split(test_size=10000, seed=3407)

# Separate into training and validation datasets
train_data = split_data['train']
val_dataset = split_data['test']

# Format the training dataset with the prompt function
train_dataset = train_data.map(formatting_prompts_func, batched=True)


In [ ]:
#print a smaple training example
train_dataset['text'][0]

# SFT

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 5000,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
!zip -r lora_model.zip lora_model

In [ ]:
from google.colab import files
files.download("lora_model.zip")

# Inference

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
import zipfile

# Path to the .zip file
zip_file_path = "/content/lora_model.zip"  # Replace with your file name

# Extract the .zip file
extracted_folder = "/content"  # Folder to extract the contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"Model extracted to: {extracted_folder}")

In [ ]:
if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
# Define parameters
batch_size = 32  # Adjust this value based on available GPU memory
test_dataset = dataset['test']
num_samples = len(test_dataset['question'])  # Set dynamically based on dataset size


# Pre-generate all input prompts for the test samples
input_prompts = [
    prompt.format(ques, ans, sol, "")  # "" for the output, allowing model to generate True/False
    for ques, ans, sol in zip(test_dataset['question'], test_dataset['answer'], test_dataset['solution'])
]

# Prepare storage for responses and true labels
responses = []
true_labels = test_dataset['is_correct']  # Assumes 'is_correct' column contains boolean values

# Process in batches
for i in range(0, num_samples, batch_size):
    # Select batch of prompts
    batch_prompts = input_prompts[i:i + batch_size]

    # Tokenize and prepare inputs for model, move to GPU
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True).to("cuda")
    input_shape = inputs['input_ids'].shape[1]  # The input token length for slicing output

    # Run inference
    outputs = model.generate(**inputs, max_new_tokens=5, use_cache=True)

    # Decode each output starting from input length to capture only the generated part
    batch_responses = tokenizer.batch_decode(
        [output[input_shape:] for output in outputs], skip_special_tokens=True
    )

    # Store the batch responses
    responses.extend(batch_responses)

    print(f"Processed batch {i // batch_size + 1}/{(num_samples + batch_size - 1) // batch_size}")

print("Inference completed for all samples.")

# At this point, 'responses' contains all generated responses, and 'true_labels' has the true values

In [ ]:
import pandas as pd

# Convert generated responses to boolean values
# Assumes `responses` is a list of lists with each inner list containing a single response string
# Convert to boolean by checking if it's "True"

predictions = [resp.strip() == 'True' for resp in responses]

# 1) Calculate accuracy
accuracy = sum(pred == label for pred, label in zip(predictions, true_labels)) / len(predictions)
print(f"Accuracy: {accuracy:.2f}")

# 2) Create a DataFrame and save to CSV
df_output = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save to CSV in Colab or Kaggle
csv_path = 'predictions2.csv'
df_output.to_csv(csv_path, index=False)

print(f"CSV file saved as '{csv_path}' in the current directory.")

In [ ]:
df_output['is_correct'].value_counts()